# Introduction to Deep Learning: Homework 5 & 6

**Nathan Inkawhich**

**[Duke Community Standard](http://integrity.duke.edu/standard.html): By typing your name below, you are certifying that you have adhered to the Duke Community Standard in completing this assignment.**

Name: Nathan Inkawhich

## Problem 4: SARSA Q-Learning (30 points)

In [1]:
import random
import gym
import math
import numpy as np
import tensorflow as tf
from collections import deque

### Problem 4a: Implement a deep Q-learning approach to the cart pole problem (was done in class) using an epsilon-Greedy approach

In [21]:
# Based on: https://gym.openai.com/evaluations/eval_EIcM1ZBnQW2LBaFN6FY65g/
class DQNCartPoleSolver_QLearning():
    def __init__(self, n_episodes=1000, n_win_ticks=195, max_env_steps=None, gamma=1.0, epsilon=1.0, epsilon_min=0.01, epsilon_log_decay=0.995, alpha=0.01, alpha_decay=0.01, batch_size=64, monitor=False, quiet=False):
        self.memory = deque(maxlen=100000)
        self.env = gym.make('CartPole-v0')
        if monitor: self.env = gym.wrappers.Monitor(self.env, '../data/cartpole-1', force=True)
        self.gamma = gamma
        self.epsilon = epsilon
        self.epsilon_min = epsilon_min
        self.epsilon_decay = epsilon_log_decay
        self.alpha = alpha
        self.alpha_decay = alpha_decay
        self.n_episodes = n_episodes
        self.n_win_ticks = n_win_ticks
        self.batch_size = batch_size
        self.quiet = quiet
        if max_env_steps is not None: self.env._max_episode_steps = max_env_steps

        # Init model
        self.state_ = tf.placeholder(tf.float32, shape=[None, 4])
        h = tf.layers.dense(self.state_, units=24, activation=tf.nn.tanh)
        h = tf.layers.dense(h, units=48, activation=tf.nn.tanh)
        self.Q = tf.layers.dense(h, units=2)
        
        self.Q_ = tf.placeholder(tf.float32, shape=[None, 2])
        loss = tf.losses.mean_squared_error(self.Q_, self.Q)
        self.global_step = tf.Variable(0, name='global_step', trainable=False)
        lr = tf.train.exponential_decay(0.01, self.global_step, 0.995, 1)
        self.train_step = tf.train.AdamOptimizer(lr).minimize(loss, global_step=self.global_step)
        
        self.sess = tf.Session()
        self.sess.run(tf.global_variables_initializer())

    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    def choose_action(self, state, epsilon):
        return self.env.action_space.sample() if (np.random.random() <= epsilon) else np.argmax(self.sess.run(self.Q, feed_dict={self.state_: state}))

    def get_epsilon(self, t):
        return max(self.epsilon_min, min(self.epsilon, 1.0 - math.log10((t + 1) * self.epsilon_decay)))

    def preprocess_state(self, state):
        return np.reshape(state, [1, 4])

    # Function to train Q network after every episode.
    def replay(self, batch_size):
        x_batch, y_batch = [], []
        # Sample a minibatch of (state,action,reward,state',done) samples from memory. Here
        #    memory is a queue of the last 100K iterations. Note, all of these samples are
        #    not necessarily from the most recent episode but they are from recent episodes.
        minibatch = random.sample(self.memory, min(len(self.memory), batch_size))
        
        # For each (state,action,reward,state',done) snapshot in the minibatch
        for state, action, reward, next_state, done in minibatch:
            # Forward pass the state through Q network to get output scores of each action. 
            #    y_target = [[a1_score, a2_score]]
            y_target = self.sess.run(self.Q, feed_dict={self.state_: state})
            
            # For the action we took, update the reward as either the reward from finishing
            #    the game, or the reward plus the discounted future reward
            y_target[0][action] = reward if done else reward + self.gamma * np.max(self.sess.run(self.Q, feed_dict={self.state_: next_state})[0])
            
            # Format the inputs to the network so we can train in supervised way.
            x_batch.append(state[0])
            y_batch.append(y_target[0])
        
        # Run a training step on this minibatch of sequences
        self.sess.run(self.train_step, feed_dict={self.state_: np.array(x_batch), self.Q_: np.array(y_batch)})

        # Decay the epsilon by a small amount because we just learned something. In beginning,
        #    epsilon is high while the agent explores and at the end it should be low so 
        #    we can follow our "optimal" learned policy
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

    def run(self):
        scores = deque(maxlen=100)

        for e in range(self.n_episodes):
            # Run a full episode, meaning start a fresh game and have the agent
            #    try to balance the pole
            state = self.preprocess_state(self.env.reset())
            done = False
            i = 0
            while not done:
                # Uncomment to render graphics
                #if e % 100 == 0 and not self.quiet:
                #    self.env.render()
                
                # Step 1:
                # Choose action with epsilon-greedy. Note, epsilon is decaying here.
                #   In one case we just sample a random action from the action space and
                #   in the other case we forward pass the state through self.Q and greedily
                #   select the action with the largest value.
                action = self.choose_action(state, self.get_epsilon(e))
                
                # Step 2:
                # Take the action and observe the next state and reward
                next_state, reward, done, _ = self.env.step(action)
                
                # Step 3:
                # Store this (state,action,reward,state',done) sequence into a memory buffer
                next_state = self.preprocess_state(next_state)
                self.remember(state, action, reward, next_state, done)
                
                # Step 4: 
                # Perform state transition into next state
                state = next_state
                i += 1

            # Record how many timesteps the agent was alive for. This is a buffer of len 100
            #    so it only keeps the scores from the last 100 iterations
            scores.append(i)
            
            # Average the scores from the last 100 episodes
            mean_score = np.mean(scores)
            
            # Check for success
            if mean_score >= self.n_win_ticks and e >= 100:
                if not self.quiet: print('Ran {} episodes. Solved after {} trials ✔'.format(e, e - 100))
                return e - 100
            
            # Output progress
            if e % 100 == 0 and not self.quiet:
                print('[Episode {}] - Mean survival time over last 100 episodes was {} ticks.'.format(e, mean_score))
            
            # After every episode, train the Q network.
            self.replay(self.batch_size)
        
        if not self.quiet: print('Did not solve after {} episodes 😞'.format(e))
        return e

if __name__ == '__main__':
    agent = DQNCartPoleSolver_QLearning(n_episodes=1500)
    agent.run()

[Episode 0] - Mean survival time over last 100 episodes was 11.0 ticks.
[Episode 100] - Mean survival time over last 100 episodes was 30.68 ticks.
[Episode 200] - Mean survival time over last 100 episodes was 57.62 ticks.
[Episode 300] - Mean survival time over last 100 episodes was 76.54 ticks.
[Episode 400] - Mean survival time over last 100 episodes was 94.31 ticks.
[Episode 500] - Mean survival time over last 100 episodes was 76.39 ticks.
[Episode 600] - Mean survival time over last 100 episodes was 120.25 ticks.
[Episode 700] - Mean survival time over last 100 episodes was 145.88 ticks.
[Episode 800] - Mean survival time over last 100 episodes was 186.74 ticks.
[Episode 900] - Mean survival time over last 100 episodes was 176.81 ticks.
Ran 919 episodes. Solved after 819 trials ✔


### Problem 4b: Implement a SARSA approach with a deep network to solve the cart pole problem using an epsilon-Greedy approach

In [26]:
# Based on: https://gym.openai.com/evaluations/eval_EIcM1ZBnQW2LBaFN6FY65g/
class DQNCartPoleSolver_SARSA():
    def __init__(self, n_episodes=1000, n_win_ticks=195, max_env_steps=None, gamma=1.0, epsilon=1.0, epsilon_min=0.01, epsilon_log_decay=0.995, alpha=0.01, alpha_decay=0.01, batch_size=64, monitor=False, quiet=False):
        self.memory = deque(maxlen=100000)
        self.env = gym.make('CartPole-v0')
        if monitor: self.env = gym.wrappers.Monitor(self.env, '../data/cartpole-1', force=True)
        self.gamma = gamma
        self.epsilon = epsilon
        self.epsilon_min = epsilon_min
        self.epsilon_decay = epsilon_log_decay
        self.alpha = alpha
        self.alpha_decay = alpha_decay
        self.n_episodes = n_episodes
        self.n_win_ticks = n_win_ticks
        self.batch_size = batch_size
        self.quiet = quiet
        if max_env_steps is not None: self.env._max_episode_steps = max_env_steps

        # Init model
        self.state_ = tf.placeholder(tf.float32, shape=[None, 4])
        h = tf.layers.dense(self.state_, units=24, activation=tf.nn.tanh)
        h = tf.layers.dense(h, units=48, activation=tf.nn.tanh)
        self.Q = tf.layers.dense(h, units=2)
        
        self.Q_ = tf.placeholder(tf.float32, shape=[None, 2])
        loss = tf.losses.mean_squared_error(self.Q_, self.Q)
        self.global_step = tf.Variable(0, name='global_step', trainable=False)
        lr = tf.train.exponential_decay(0.01, self.global_step, 0.995, 1)
        self.train_step = tf.train.AdamOptimizer(lr).minimize(loss, global_step=self.global_step)
        
        self.sess = tf.Session()
        self.sess.run(tf.global_variables_initializer())

    def remember(self, state, action, reward, next_state, next_action, done):
        self.memory.append((state, action, reward, next_state, next_action, done))

    def choose_action(self, state, epsilon):
        return self.env.action_space.sample() if (np.random.random() <= epsilon) else np.argmax(self.sess.run(self.Q, feed_dict={self.state_: state}))

    def get_epsilon(self, t):
        return max(self.epsilon_min, min(self.epsilon, 1.0 - math.log10((t + 1) * self.epsilon_decay)))

    def preprocess_state(self, state):
        return np.reshape(state, [1, 4])

    # Function to train Q network after every episode.
    def replay(self, batch_size):
        x_batch, y_batch = [], []
        # Sample a minibatch of (state,action,reward,state',done) samples from memory. Here
        #    memory is a queue of the last 100K iterations. Note, all of these samples are
        #    not necessarily from the most recent episode but they are from recent episodes.
        minibatch = random.sample(self.memory, min(len(self.memory), batch_size))
        
        # For each (state,action,reward,state',done) snapshot in the minibatch
        for state, action, reward, next_state, next_action, done in minibatch:
            # Forward pass the state through Q network to get output scores of each action. 
            #    y_target = [[a1_score, a2_score]]
            y_target = self.sess.run(self.Q, feed_dict={self.state_: state})
            
            # SARSA
            # For the action we took, update the reward as either the reward from finishing
            #    the game, or the reward plus the discounted future reward
            #OLD: y_target[0][action] = reward if done else reward + self.gamma * np.max(self.sess.run(self.Q, feed_dict={self.state_: next_state})[0])
            y_target[0][action] = reward if done else reward + self.gamma * self.sess.run(self.Q, feed_dict={self.state_: next_state})[0][next_action]
            
            # Format the inputs to the network so we can train in supervised way.
            x_batch.append(state[0])
            y_batch.append(y_target[0])
        
        # Run a training step on this minibatch of sequences
        self.sess.run(self.train_step, feed_dict={self.state_: np.array(x_batch), self.Q_: np.array(y_batch)})

        # Decay the epsilon by a small amount because we just learned something. In beginning,
        #    epsilon is high while the agent explores and at the end it should be low so 
        #    we can follow our "optimal" learned policy
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

    def run(self):
        scores = deque(maxlen=100)

        for e in range(self.n_episodes):
            # Run a full episode, meaning start a fresh game and have the agent
            #    try to balance the pole
            state = self.preprocess_state(self.env.reset())
            
            # SARSA: Choose initial action to get started
            action = self.choose_action(state, self.get_epsilon(e))
            
            done = False
            i = 0
            while not done:
                # Uncomment to render graphics
                #if e % 100 == 0 and not self.quiet:
                #    self.env.render()
                
                # Take the action and observe the next state and reward
                next_state, reward, done, _ = self.env.step(action)
                next_state = self.preprocess_state(next_state)
                
                # SARSA: Choose next action based on next state. This is important because
                #     Q-fxn will be updated with this info.
                action_1 = self.choose_action(next_state, self.get_epsilon(e))
                
                # Store this (state,action,reward,state',action',done) sequence into a memory buffer
                self.remember(state, action, reward, next_state, action_1, done)
                
                # Perform state transition into next state
                state = next_state
                action=action_1
                i += 1

            # Record how many timesteps the agent was alive for. This is a buffer of len 100
            #    so it only keeps the scores from the last 100 iterations
            scores.append(i)
            
            # Average the scores from the last 100 episodes
            mean_score = np.mean(scores)
            
            # Check for success
            if mean_score >= self.n_win_ticks and e >= 100:
                if not self.quiet: print('Ran {} episodes. Solved after {} trials ✔'.format(e, e - 100))
                return e - 100
            
            # Output progress
            if e % 100 == 0 and not self.quiet:
                print('[Episode {}] - Mean survival time over last 100 episodes was {} ticks.'.format(e, mean_score))
            
            # After every episode, train the Q network.
            self.replay(self.batch_size)
        
        if not self.quiet: print('Did not solve after {} episodes 😞'.format(e))
        return e

if __name__ == '__main__':
    agent = DQNCartPoleSolver_SARSA(n_episodes=1500)
    agent.run()

[Episode 0] - Mean survival time over last 100 episodes was 13.0 ticks.
[Episode 100] - Mean survival time over last 100 episodes was 23.55 ticks.
[Episode 200] - Mean survival time over last 100 episodes was 54.33 ticks.
[Episode 300] - Mean survival time over last 100 episodes was 180.84 ticks.
[Episode 400] - Mean survival time over last 100 episodes was 186.0 ticks.
[Episode 500] - Mean survival time over last 100 episodes was 194.64 ticks.
Ran 518 episodes. Solved after 418 trials ✔


### Problem 4c: Evaluate the impact of $\gamma$ in the cart pole problem. How important is this parameter? How does it affect stability and learning?

In [32]:
for gamma in [0, .2, .4, .6, .8, 1.]:
    print("##################\ngamma={}\n##################\n".format(gamma))
    agent = DQNCartPoleSolver_QLearning(n_episodes=2000, gamma=gamma)
    agent.run()

##################
gamma=0
##################

[Episode 0] - Mean survival time over last 100 episodes was 12.0 ticks.
[Episode 100] - Mean survival time over last 100 episodes was 27.94 ticks.
[Episode 200] - Mean survival time over last 100 episodes was 9.46 ticks.
[Episode 300] - Mean survival time over last 100 episodes was 9.58 ticks.
[Episode 400] - Mean survival time over last 100 episodes was 11.06 ticks.
[Episode 500] - Mean survival time over last 100 episodes was 11.87 ticks.
[Episode 600] - Mean survival time over last 100 episodes was 11.56 ticks.
[Episode 700] - Mean survival time over last 100 episodes was 14.06 ticks.
[Episode 800] - Mean survival time over last 100 episodes was 14.69 ticks.
[Episode 900] - Mean survival time over last 100 episodes was 15.87 ticks.
[Episode 1000] - Mean survival time over last 100 episodes was 10.96 ticks.
[Episode 1100] - Mean survival time over last 100 episodes was 26.64 ticks.
[Episode 1200] - Mean survival time over last 100 episod

[Episode 300] - Mean survival time over last 100 episodes was 106.26 ticks.
[Episode 400] - Mean survival time over last 100 episodes was 105.88 ticks.
[Episode 500] - Mean survival time over last 100 episodes was 119.61 ticks.
[Episode 600] - Mean survival time over last 100 episodes was 119.06 ticks.
[Episode 700] - Mean survival time over last 100 episodes was 116.43 ticks.
[Episode 800] - Mean survival time over last 100 episodes was 165.1 ticks.
[Episode 900] - Mean survival time over last 100 episodes was 159.63 ticks.
[Episode 1000] - Mean survival time over last 100 episodes was 100.44 ticks.
[Episode 1100] - Mean survival time over last 100 episodes was 102.21 ticks.
Ran 1197 episodes. Solved after 1097 trials ✔


In [33]:
for gamma in [0, .2, .4, .6, .8, 1.]:
    print("##################\nSARSA gamma={}\n##################\n".format(gamma))
    agent = DQNCartPoleSolver_SARSA(n_episodes=2000, gamma=gamma)
    agent.run()

##################
SARSA gamma=0
##################

[Episode 0] - Mean survival time over last 100 episodes was 22.0 ticks.
[Episode 100] - Mean survival time over last 100 episodes was 14.54 ticks.
[Episode 200] - Mean survival time over last 100 episodes was 9.42 ticks.
[Episode 300] - Mean survival time over last 100 episodes was 9.22 ticks.
[Episode 400] - Mean survival time over last 100 episodes was 9.36 ticks.
[Episode 500] - Mean survival time over last 100 episodes was 9.48 ticks.
[Episode 600] - Mean survival time over last 100 episodes was 9.47 ticks.
[Episode 700] - Mean survival time over last 100 episodes was 10.69 ticks.
[Episode 800] - Mean survival time over last 100 episodes was 10.14 ticks.
[Episode 900] - Mean survival time over last 100 episodes was 9.96 ticks.
[Episode 1000] - Mean survival time over last 100 episodes was 9.54 ticks.
[Episode 1100] - Mean survival time over last 100 episodes was 9.88 ticks.
[Episode 1200] - Mean survival time over last 100 episod

[Episode 200] - Mean survival time over last 100 episodes was 145.55 ticks.
[Episode 300] - Mean survival time over last 100 episodes was 123.39 ticks.
[Episode 400] - Mean survival time over last 100 episodes was 169.35 ticks.
[Episode 500] - Mean survival time over last 100 episodes was 174.99 ticks.
[Episode 600] - Mean survival time over last 100 episodes was 185.78 ticks.
[Episode 700] - Mean survival time over last 100 episodes was 193.16 ticks.
Ran 714 episodes. Solved after 614 trials ✔


**Answer here**

### Problem 4d: Evaluate the impact of $\epsilon$ on the learning over the feasible range (0-1). What values seem reasonable?

In [35]:
for eps in [0., .2, .4, .6, .8, 1.]:
    print("##################\nepsilon={}\n##################\n".format(eps))
    agent = DQNCartPoleSolver_QLearning(n_episodes=1500, epsilon=eps, epsilon_min=0.0, epsilon_log_decay=0.995)
    agent.run()

##################
epsilon=0.0
##################

[Episode 0] - Mean survival time over last 100 episodes was 10.0 ticks.
[Episode 100] - Mean survival time over last 100 episodes was 9.2 ticks.
[Episode 200] - Mean survival time over last 100 episodes was 9.37 ticks.
[Episode 300] - Mean survival time over last 100 episodes was 9.44 ticks.
[Episode 400] - Mean survival time over last 100 episodes was 9.39 ticks.
[Episode 500] - Mean survival time over last 100 episodes was 9.32 ticks.
[Episode 600] - Mean survival time over last 100 episodes was 9.35 ticks.
[Episode 700] - Mean survival time over last 100 episodes was 9.49 ticks.
[Episode 800] - Mean survival time over last 100 episodes was 53.19 ticks.
[Episode 900] - Mean survival time over last 100 episodes was 43.38 ticks.
[Episode 1000] - Mean survival time over last 100 episodes was 91.45 ticks.
[Episode 1100] - Mean survival time over last 100 episodes was 103.22 ticks.
[Episode 1200] - Mean survival time over last 100 episode

In [36]:
for eps in [0., .2, .4, .6, .8, 1.]:
    print("##################\nSARSA epsilon={}\n##################\n".format(eps))
    agent = DQNCartPoleSolver_SARSA(n_episodes=1500, epsilon=eps, epsilon_min=0.0, epsilon_log_decay=0.995)
    agent.run()

##################
SARSA epsilon=0.0
##################

[Episode 0] - Mean survival time over last 100 episodes was 200.0 ticks.
[Episode 100] - Mean survival time over last 100 episodes was 69.85 ticks.
[Episode 200] - Mean survival time over last 100 episodes was 107.97 ticks.
[Episode 300] - Mean survival time over last 100 episodes was 57.7 ticks.
[Episode 400] - Mean survival time over last 100 episodes was 87.47 ticks.
[Episode 500] - Mean survival time over last 100 episodes was 180.38 ticks.
[Episode 600] - Mean survival time over last 100 episodes was 184.75 ticks.
[Episode 700] - Mean survival time over last 100 episodes was 181.93 ticks.
[Episode 800] - Mean survival time over last 100 episodes was 180.48 ticks.
[Episode 900] - Mean survival time over last 100 episodes was 168.03 ticks.
[Episode 1000] - Mean survival time over last 100 episodes was 132.44 ticks.
[Episode 1100] - Mean survival time over last 100 episodes was 140.11 ticks.
[Episode 1200] - Mean survival time 

**Answer here**

### Problem 4e: Pick one other small agent from the OpenAI gym and apply the same techniques

In [10]:
#env = gym.make("Acrobot-v1")
#env = gym.make("Pong-v0")
env = gym.make("MountainCar-v0")

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.


In [11]:
env.observation_space

Box(2,)

In [12]:
env.action_space

Discrete(3)

In [26]:
env.close()

#### Problem 4e: Q-Learning Solver

In [5]:
class DQNMountainCarSolver():
    def __init__(self, n_episodes=1000, n_win_ticks=195, max_env_steps=None, gamma=1.0, epsilon=1.0, epsilon_min=0.1, epsilon_log_decay=0.99, alpha=0.01, alpha_decay=0.01, batch_size=64, monitor=False, quiet=False):
        self.memory = deque(maxlen=10000)
        self.env = gym.make('MountainCar-v0')
        if monitor: self.env = gym.wrappers.Monitor(self.env, '../data/mountaincar-1', force=True)
        self.gamma = gamma
        self.lr = 0.001
        self.epsilon = epsilon
        #self.epsilon = 0.3
        self.epsilon_min = epsilon_min
        self.epsilon_decay = epsilon_log_decay
        self.alpha = alpha
        self.alpha_decay = alpha_decay
        self.n_episodes = n_episodes
        self.n_win_ticks = n_win_ticks
        self.batch_size = batch_size
        self.quiet = quiet
        if max_env_steps is not None: self.env._max_episode_steps = max_env_steps

        # Init Q model
        self.state_ = tf.placeholder(tf.float32, shape=[None, 2])
        h = tf.layers.dense(self.state_, units=50, activation=tf.nn.tanh, kernel_initializer=tf.truncated_normal_initializer)
        #h = tf.layers.dense(self.state_, units=100, activation=None,kernel_initializer=tf.truncated_normal_initializer)
        #h = tf.layers.dense(self.state_, units=50, activation=tf.nn.tanh,kernel_initializer=tf.truncated_normal_initializer)
        #h = tf.layers.dense(h, units=50, activation=tf.nn.tanh)
        self.Q = tf.layers.dense(h, units=3)
        
        self.Q_ = tf.placeholder(tf.float32, shape=[None, 3])
        loss = tf.losses.mean_squared_error(self.Q_, self.Q)
        #self.global_step = tf.Variable(0, name='global_step', trainable=False)
        #lr = tf.train.exponential_decay(0.001, self.global_step, 0.995, 1)
        #self.train_step = tf.train.AdamOptimizer().minimize(loss, global_step=self.global_step)
        self.learning_rate = tf.placeholder(tf.float32, shape=[])
        self.train_step = tf.train.GradientDescentOptimizer(learning_rate=self.learning_rate).minimize(loss)
        
        self.sess = tf.Session()
        self.sess.run(tf.global_variables_initializer())

    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    def choose_action(self, state, epsilon):
        return self.env.action_space.sample() if (np.random.random() <= epsilon) else np.argmax(self.sess.run(self.Q, feed_dict={self.state_: state, self.learning_rate: self.lr}))

    def get_epsilon(self, t):
        #return max(self.epsilon_min, min(self.epsilon, 1.0 - math.log10((t + 1) * self.epsilon_decay)))
        return self.epsilon
    
    def preprocess_state(self, state):
        return np.reshape(state, [1, 2])

    # Function to train Q network after every episode.
    def replay(self, batch_size):
        x_batch, y_batch = [], []
        # Sample a minibatch of (state,action,reward,state',done) samples from memory. Here
        #    memory is a queue of the last 100K iterations. Note, all of these samples are
        #    not necessarily from the most recent episode but they are from recent episodes.
        minibatch = random.sample(self.memory, min(len(self.memory), batch_size))
        
        # For each (state,action,reward,state',done) snapshot in the minibatch
        for state, action, reward, next_state, done in minibatch:
            # Forward pass the state through Q network to get output scores of each action. 
            y_target = self.sess.run(self.Q, feed_dict={self.state_: state, self.learning_rate: self.lr})
            
            # For the action we took, update the reward as either the reward from finishing
            #    the game, or the reward plus the discounted future reward
            y_target[0][action] = reward if done else reward + self.gamma * np.max(self.sess.run(self.Q, feed_dict={self.state_: next_state, self.learning_rate: self.lr})[0])
            
            # Format the inputs to the network so we can train in supervised way.
            x_batch.append(state[0])
            y_batch.append(y_target[0])
        
        # Run a training step on this minibatch of sequences
        self.sess.run(self.train_step, feed_dict={self.state_: np.array(x_batch), self.Q_: np.array(y_batch), self.learning_rate: self.lr})

        # Decay the epsilon by a small amount because we just learned something. In beginning,
        #    epsilon is high while the agent explores and at the end it should be low so 
        #    we can follow our "optimal" learned policy
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

    def run(self):
        scores = deque(maxlen=100)
        num_wins = 0
        steps = 300
        
        for e in range(self.n_episodes):
        #for e in range(1):
            # Run a full episode, meaning start a fresh game and have the agent
            #    try to balance the pole
            state = self.preprocess_state(self.env.reset())
            done = False
            i = 0
            max_pos = -100
            
            #while not done:
            for s in range(steps):
                # Uncomment to render graphics
                #if e % 100 == 0 and not self.quiet:
                #    self.env.render()
                
                # Step 1:
                # Choose action with epsilon-greedy. Note, epsilon is decaying here.
                #   In one case we just sample a random action from the action space and
                #   in the other case we forward pass the state through self.Q and greedily
                #   select the action with the largest value.
                action = self.choose_action(state, self.get_epsilon(e))
                
                # Step 2:
                # Take the action and observe the next state and reward
                next_state, reward, done, _ = self.env.step(action)
                
                # Step 2.1: Modify reward signal to encourage faster learning.
                #    Here we adjust reward based on car position and for task completion.
                #    Note, next_state[0] is the car's position. Inspired by
                #    https://medium.com/@ts1829/solving-mountain-car-with-q-learning-b77bf71b1de2
                #print("Next state: {}; reward: {}; done: {}".format(next_state,reward,done))
                reward = next_state[0] - 0.5
                if next_state[0] >= 0.5:
                    reward += 1
                    
                # update best position seen
                if next_state[0] >= max_pos:
                    max_pos = next_state[0]
                
                # if done
                if done:
                    # if we have ended in a success, we will update some learning hyperparams
                    if next_state[0] >= 0.5:
                        # Decay epsilon
                        self.epsilon *= 0.95
                        # Decay learning rate
                        self.lr *= 0.9
                        num_wins += 1
                
                # Step 3:
                # Store this (state,action,reward,state',done) sequence into a memory buffer
                next_state = self.preprocess_state(next_state)
                self.remember(state, action, reward, next_state, done)
                
                # Step 4: 
                # Perform state transition into next state
                state = next_state
                i += 1
                
            #"""
            # Record the final position of the cart. This is a buffer of len 100
            #    so it only keeps the positions from the last 100 iterations
            #scores.append(next_state[0])
            scores.append(max_pos)
            
            # Average the scores from the last 100 episodes
            mean_score = np.mean(scores)
            max_score = np.max(scores)
            
            # Check for success
            if max_score >= 0.5 and e >= 100:
                if not self.quiet: print('Ran {} episodes. Solved after {} trials ✔'.format(e, e - 100))
                return e - 100
            
            # Output progress
            if e % 20 == 0 and not self.quiet:
                print('[Episode {}] - Mean/Max score {}/{}. Num wins: {}. eps = {}. lr = {}'.format(e, mean_score,max_score, num_wins,self.epsilon,self.lr))
            
            # After every episode, train the Q network.
            self.replay(self.batch_size)
            #"""
        
        if not self.quiet: print('Did not solve after {} episodes 😞'.format(e))
        return e

if __name__ == '__main__':
    agent = DQNMountainCarSolver(n_episodes=3000,gamma=.99,epsilon=0.9, epsilon_min=0.01, epsilon_log_decay=0.999)
    agent.run()

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
[Episode 0] - Mean/Max score -0.4474913144874446/-0.4474913144874446. Num wins: 0. eps = 0.9. lr = 0.001
[Episode 20] - Mean/Max score -0.34452924208861385/-0.13733566059288155. Num wins: 0. eps = 0.8821699783465812. lr = 0.001
[Episode 40] - Mean/Max score -0.36000046946886083/-0.13733566059288155. Num wins: 0. eps = 0.8646931896622304. lr = 0.001
[Episode 60] - Mean/Max score -0.35475116904265913/-0.13733566059288155. Num wins: 0. eps = 0.8475626360008509. lr = 0.001
[Episode 80] - Mean/Max score -0.3584701600248111/-0.13733566059288155. Num wins: 0. eps = 0.8307714580536021. lr = 0.001
[Episode 100] - Mean/Max score -0.3521744009452288/-0.13733566059288155. Num wins: 0. eps = 0.8143129324023378. lr = 0.001
[Episode 120] - Mean/Max score -0.35871806450134386/-0.15023361034718635. Num wins: 0. eps = 0.7981804688274572. lr = 0.001
[Episode 140] - Mean/Max score -0.36210102103765623/-0.150

#### Problem 4e: SARSA Solver

In [13]:
class DQNMountainCarSolver_SARSA():
    def __init__(self, n_episodes=1000, n_win_ticks=195, max_env_steps=None, gamma=1.0, epsilon=1.0, epsilon_min=0.1, epsilon_log_decay=0.99, alpha=0.01, alpha_decay=0.01, batch_size=64, monitor=False, quiet=False):
        self.memory = deque(maxlen=10000)
        self.env = gym.make('MountainCar-v0')
        if monitor: self.env = gym.wrappers.Monitor(self.env, '../data/mountaincar-1', force=True)
        self.gamma = gamma
        self.lr = 0.001
        self.epsilon = epsilon
        #self.epsilon = 0.3
        self.epsilon_min = epsilon_min
        self.epsilon_decay = epsilon_log_decay
        self.alpha = alpha
        self.alpha_decay = alpha_decay
        self.n_episodes = n_episodes
        self.n_win_ticks = n_win_ticks
        self.batch_size = batch_size
        self.quiet = quiet
        if max_env_steps is not None: self.env._max_episode_steps = max_env_steps

        # Init Q model
        self.state_ = tf.placeholder(tf.float32, shape=[None, 2])
        h = tf.layers.dense(self.state_, units=50, activation=tf.nn.tanh, kernel_initializer=tf.truncated_normal_initializer)
        #h = tf.layers.dense(self.state_, units=100, activation=None,kernel_initializer=tf.truncated_normal_initializer)
        #h = tf.layers.dense(self.state_, units=50, activation=tf.nn.tanh,kernel_initializer=tf.truncated_normal_initializer)
        #h = tf.layers.dense(h, units=50, activation=tf.nn.tanh)
        self.Q = tf.layers.dense(h, units=3)
        
        self.Q_ = tf.placeholder(tf.float32, shape=[None, 3])
        loss = tf.losses.mean_squared_error(self.Q_, self.Q)
        #self.global_step = tf.Variable(0, name='global_step', trainable=False)
        #lr = tf.train.exponential_decay(0.001, self.global_step, 0.995, 1)
        #self.train_step = tf.train.AdamOptimizer().minimize(loss, global_step=self.global_step)
        self.learning_rate = tf.placeholder(tf.float32, shape=[])
        self.train_step = tf.train.GradientDescentOptimizer(learning_rate=self.learning_rate).minimize(loss)
        
        self.sess = tf.Session()
        self.sess.run(tf.global_variables_initializer())

    def remember(self, state, action, reward, next_state, next_action, done):
        self.memory.append((state, action, reward, next_state, next_action, done))

    def choose_action(self, state, epsilon):
        return self.env.action_space.sample() if (np.random.random() <= epsilon) else np.argmax(self.sess.run(self.Q, feed_dict={self.state_: state, self.learning_rate: self.lr}))

    def get_epsilon(self, t):
        #return max(self.epsilon_min, min(self.epsilon, 1.0 - math.log10((t + 1) * self.epsilon_decay)))
        return self.epsilon
    
    def preprocess_state(self, state):
        return np.reshape(state, [1, 2])

    # Function to train Q network after every episode.
    def replay(self, batch_size):
        x_batch, y_batch = [], []
        # Sample a minibatch of (state,action,reward,state',done) samples from memory. Here
        #    memory is a queue of the last 100K iterations. Note, all of these samples are
        #    not necessarily from the most recent episode but they are from recent episodes.
        minibatch = random.sample(self.memory, min(len(self.memory), batch_size))
        
        # For each (state,action,reward,state',done) snapshot in the minibatch
        for state, action, reward, next_state, next_action, done in minibatch:
            # Forward pass the state through Q network to get output scores of each action. 
            y_target = self.sess.run(self.Q, feed_dict={self.state_: state, self.learning_rate: self.lr})
            
            # SARSA
            # For the action we took, update the reward as either the reward from finishing
            #    the game, or the reward plus the discounted future reward
            # OLD: y_target[0][action] = reward if done else reward + self.gamma * np.max(self.sess.run(self.Q, feed_dict={self.state_: next_state, self.learning_rate: self.lr})[0])
            y_target[0][action] = reward if done else reward + self.gamma * self.sess.run(self.Q, feed_dict={self.state_: next_state, self.learning_rate: self.lr})[0][next_action]
            
            # Format the inputs to the network so we can train in supervised way.
            x_batch.append(state[0])
            y_batch.append(y_target[0])
        
        # Run a training step on this minibatch of sequences
        self.sess.run(self.train_step, feed_dict={self.state_: np.array(x_batch), self.Q_: np.array(y_batch), self.learning_rate: self.lr})

        # Decay the epsilon by a small amount because we just learned something. In beginning,
        #    epsilon is high while the agent explores and at the end it should be low so 
        #    we can follow our "optimal" learned policy
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

    def run(self):
        scores = deque(maxlen=100)
        num_wins = 0
        steps = 300
        
        for e in range(self.n_episodes):
            
            # Pick an initial state and action
            state = self.preprocess_state(self.env.reset())
            action = self.choose_action(state, self.get_epsilon(e))
                
            done = False
            i = 0
            max_pos = -100
            
            for s in range(steps):
                # Uncomment to render graphics
                #if e % 100 == 0 and not self.quiet:
                #    self.env.render()
                
                # Take the action and observe the next state and reward
                next_state, reward, done, _ = self.env.step(action)
                
                # Modify reward signal to encourage faster learning.
                #    Here we adjust reward based on car position and for task completion.
                #    Note, next_state[0] is the car's position. Inspired by
                #    https://medium.com/@ts1829/solving-mountain-car-with-q-learning-b77bf71b1de2
                #print("Next state: {}; reward: {}; done: {}".format(next_state,reward,done))
                reward = next_state[0] - 0.5
                if next_state[0] >= 0.5:
                    reward += 1
                    
                # update best position seen
                if next_state[0] >= max_pos:
                    max_pos = next_state[0]
                
                # if done
                if done:
                    # if we have ended in a success, we will update some learning hyperparams
                    if next_state[0] >= 0.5:
                        # Decay epsilon
                        self.epsilon *= 0.95
                        # Decay learning rate
                        self.lr *= 0.9
                        num_wins += 1
                
                # Using next state pick next action for SARSA
                next_state = self.preprocess_state(next_state)
                next_action = self.choose_action(next_state, self.get_epsilon(e))
                
                # Store this (state,action,reward,state',done) sequence into a memory buffer
                self.remember(state, action, reward, next_state, next_action, done)
                
                # Step 4: 
                # Perform state transition into next state
                state = next_state
                action = next_action
                i += 1
                
            #"""
            # Record the final position of the cart. This is a buffer of len 100
            #    so it only keeps the positions from the last 100 iterations
            #scores.append(next_state[0])
            scores.append(max_pos)
            
            # Average the scores from the last 100 episodes
            mean_score = np.mean(scores)
            max_score = np.max(scores)
            
            # Check for success
            if max_score >= 0.5 and e >= 100:
                if not self.quiet: print('Ran {} episodes. Solved after {} trials ✔'.format(e, e - 100))
                return e - 100
            
            # Output progress
            if e % 20 == 0 and not self.quiet:
                print('[Episode {}] - Mean/Max score {}/{}. Num wins: {}. eps = {}. lr = {}'.format(e, mean_score,max_score, num_wins,self.epsilon,self.lr))
            
            # After every episode, train the Q network.
            self.replay(self.batch_size)
            #"""
        
        if not self.quiet: print('Did not solve after {} episodes 😞'.format(e))
        return e

if __name__ == '__main__':
    agent = DQNMountainCarSolver_SARSA(n_episodes=3000,gamma=.99,epsilon=0.9, epsilon_min=0.01, epsilon_log_decay=0.999)
    agent.run()

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
[Episode 0] - Mean/Max score -0.47125564781239215/-0.47125564781239215. Num wins: 0. eps = 0.9. lr = 0.001
[Episode 20] - Mean/Max score -0.3684692801096171/-0.21411975592712723. Num wins: 0. eps = 0.8821699783465812. lr = 0.001
[Episode 40] - Mean/Max score -0.3610644160333316/-0.21411975592712723. Num wins: 0. eps = 0.8646931896622304. lr = 0.001
[Episode 60] - Mean/Max score -0.36756389012880075/-0.21411975592712723. Num wins: 0. eps = 0.8475626360008509. lr = 0.001
[Episode 80] - Mean/Max score -0.3737211393262115/-0.21411975592712723. Num wins: 0. eps = 0.8307714580536021. lr = 0.001
[Episode 100] - Mean/Max score -0.36371559610665094/-0.09351564248902206. Num wins: 0. eps = 0.8143129324023378. lr = 0.001
[Episode 120] - Mean/Max score -0.35821296849413914/-0.07175527200867365. Num wins: 0. eps = 0.7981804688274572. lr = 0.001
[Episode 140] - Mean/Max score -0.3149887485762749/0.0564